In [3]:
# Try to decode which feature was selected per-trial based on firing rates of neurons
# experiment with ranges of firing rates around fixation (selection) time

%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats
from lfp_tools import (
    general as lfp_general,
    startup as lfp_startup,
    development as lfp_development,
    analysis as lfp_analysis
)
from spike_tools import (
    general as spike_general,
    analysis as spike_analysis,
)
import s3fs
import utils.behavioral_utils as behavioral_utils
import utils.spike_utils as spike_utils
import utils.classifier_utils as classifier_utils
import utils.visualization_utils as visualization_utils
from trial_splitters.random_splitter import RandomSplitter
from trial_splitters.block_splitter import BlockSplitter
from sklearn import svm

species = 'nhp'
subject = 'SA'
exp = 'WCST'
session = 20180802  # this is the session for which there are spikes at the moment. 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# grab behavioral data, spike data, trial numbers. 

fs = s3fs.S3FileSystem()

behavior_file = spike_general.get_behavior_path(subject, session)
behavior_data = pd.read_csv(fs.open(behavior_file))
valid_beh = behavior_data[behavior_data.Response.isin(["Correct", "Incorrect"])]
trial_numbers = np.unique(valid_beh.TrialNumber)
spike_times = spike_general.get_spike_times(fs, subject, session)


In [56]:
spike_general.list_session_units(fs, subject, session)

temp_units = spike_utils.get_temporal_drive_unit_ids(fs, subject, session)
print(temp_units)

ant_units = spike_utils.get_anterior_drive_unit_ids(fs, subject, session)
print(ant_units)

[ 1  2  3  4  5  6  7  9 10 11 12 13 15 17 18 19 35 39 42 43 44 45 46 47
 48 49 54 55 59]
[ 8 14 16 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 36 37 38 40 41 50
 51 52 53 56 57 58]


In [7]:
# find "fixation times". In this case, 800ms before feedback. 
pre_interval = 800
post_interval = 0
intervals = behavioral_utils.get_trial_intervals(valid_beh, "FeedbackOnset", pre_interval, post_interval)

In [ ]:
# find the spikes falling in the intervals
spike_times = spike_general.get_spike_times(fs, subject, session)
spike_by_trial_interval = spike_utils.get_spikes_by_trial_interval(spike_times, intervals)

In [32]:
# grab firing rates, spike counts, in 800ms intervals, 100ms bins
firing_rates = spike_analysis.firing_rate(spike_by_trial_interval, bins=np.arange(0, 0.9, 0.1), smoothing=1)

In [3]:
# grab Shape, Color, Pattern features for each selection
selections = behavioral_utils.get_selection_features(valid_beh)

In [5]:
selections.to_pickle(fs.open(f"l2l.pqz317.scratch/feature_selections.pickle", "wb"))


In [2]:
# load what's already been computed
feature_selections = pd.read_pickle("../data/feature_selections.pickle")
firing_rates = pd.read_pickle("../data/firing_rates.pickle")

FileNotFoundError: [Errno 2] No such file or directory: '../data/feature_selections.pickle'

In [62]:
# use a support vector machine model
clf = svm.SVC(decision_function_shape='ovo')

# Look at Spike Counts instead of Firing Rate
mode = "SpikeCounts"

# Look at 'Pattern' as a dimension
feature_dim = "Pattern"

# prep data for classification
inputs = firing_rates.rename(columns={mode: "Value"})
labels = feature_selections.rename(columns={feature_dim: "Feature"})


In [64]:
# randomly grab trials for train/test, run 20 times, split 80/20
random_splitter = RandomSplitter(labels.TrialNumber.unique(), 20, 0.2)
train_accs, test_accs, shuffled_accs, models = classifier_utils.evaluate_classifier(clf, inputs, labels, random_splitter)
print(f"Mean Train accuracy: {np.mean(train_accs)}, Test accuracy: {np.mean(test_accs)}, Shuffled accuracy: {np.mean(shuffled_accs)}")


In [ ]:
# split trials into train/test by blocks, always leave one block out for test. 
block_splitter = BlockSplitter(valid_beh)
train_accs, test_accs, shuffled_accs, models = classifier_utils.evaluate_classifier(clf, inputs, labels, block_splitter)

In [ ]:
# look across different time bins, evaluate classifier on each time bin individually
time_bins = np.arange(0.0, 0.8, 0.1)
for bin in time_bins:
    inputs_for_bin = inputs[inputs["TimeBins"] == bin]
    train_accs, test_accs, shuffled_accs, models = classifier_utils.evaluate_classifier(clf, inputs_for_bin, labels, random_splitter)
    print(f"Mean Train accuracy: {np.mean(train_accs)}, Test accuracy: {np.mean(test_accs)}, Shuffled accuracy: {np.mean(shuffled_accs)}")

In [4]:
# find "fixation times". In this case, 800ms before feedback. 
pre_interval = 1200
post_interval = 800
intervals = behavioral_utils.get_trial_intervals(valid_beh, "FeedbackOnset", pre_interval, post_interval)

In [5]:
# find the spikes falling in the intervals
spike_times = spike_general.get_spike_times(fs, subject, session)
spike_by_trial_interval = spike_utils.get_spikes_by_trial_interval(spike_times, intervals)

In [7]:
spike_by_trial_interval.to_pickle("../data/spike_by_trial_interval_1200_fb_800.pickle")

In [22]:
spike_by_trial_interval = pd.read_pickle("../data/spike_by_trial_interval_1200_fb_800.pickle")
firing_rates = spike_analysis.firing_rate(spike_by_trial_interval, bins=np.arange(0, 2.1, 0.1), smoothing=1)
firing_rates.to_pickle("../data/firing_rates_1200_fb_800.pickle")

In [2]:
feature_selections = pd.read_pickle("../data/feature_selections.pickle")
firing_rates = pd.read_pickle("../data/firing_rates_1200_fb_800.pickle")

In [3]:
# use a support vector machine model
clf = svm.SVC(decision_function_shape='ovo')

# Look at Spike Counts instead of Firing Rate
mode = "SpikeCounts"

# Look at 'Pattern' as a dimension
feature_dim = "Pattern"

# prep data for classification
inputs = firing_rates.rename(columns={mode: "Value"})
labels = feature_selections.rename(columns={feature_dim: "Feature"})

In [58]:
random_splitter = RandomSplitter(labels.TrialNumber.unique(), 20, 0.2)
ant_inputs = inputs[inputs["UnitID"].isin(ant_units)]
_, test_accs_by_bin, shuffled_accs, _, _ = classifier_utils.evaluate_classifiers_by_time_bins(
    clf, ant_inputs, labels, np.arange(0, 2, 0.1), random_splitter
)

fig, ax = plt.subplots()
visualization_utils.visualize_accuracy_across_time_bins(
    test_accs_by_bin,
    1200, 800, 100,
    ax,
    label="Anterior Units"
)
visualization_utils.visualize_accuracy_across_time_bins(
    shuffled_accs,
    1200, 800, 100,
    ax,
    label="Shuffled"
)

temp_inputs = inputs[inputs["UnitID"].isin(temp_units)]
test_accs_by_bin, shuffled_accs, _, _ = classifier_utils.evaluate_classifiers_by_time_bins(
    clf, temp_inputs, labels, np.arange(0, 2.8, 0.1), random_splitter
)

visualization_utils.visualize_accuracy_across_time_bins(
    shuffled_accs,
    1200, 800, 100,
    ax,
    label="Temporal Units"
)
ax.legend()
ax.axvline(-800, color='k')

ValueError: Found array with 0 sample(s) (shape=(0, 0)) while a minimum of 1 is required by SVC.

In [24]:
np.save("../data/accs_by_bin_block_splitter.npy", test_accs_by_bin)